# Fine-tune для Qwen2.5-VL-7B-Instruct-bnb-4bit (4bit LoRA)

Базируется на официальном ноутбуке от 🦥[Unsloth](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_VL_(7B)-Vision.ipynb)

## Подготовка датасета

In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import math
import ast

HF_TOKEN = "XXXXX"

In [2]:
# 1 Вариант - для Google Colab

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

!kaggle datasets download -d moxeeeem/aaaproject
!kaggle datasets download -d moxeeeem/aaa-data-2

!unzip -q aaaproject.zip -d ./ && rm aaaproject.zip
!unzip -q aaa-data-2.zip -d ./ && rm aaa-data-2.zip

IMG_DIR = "/content/aaa_extracted/aaa_desc_gen_data/images/"
DATA_CSV = "/content/full_dataset.csv"

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/moxeeeem/aaaproject
License(s): unknown
100% 3.66G/3.67G [00:56<00:00, 151MB/s]
100% 3.67G/3.67G [00:56<00:00, 69.2MB/s]
Dataset URL: https://www.kaggle.com/datasets/moxeeeem/aaa-data-2
License(s): unknown
 86% 93.0M/109M [00:00<00:00, 975MB/s]
100% 109M/109M [00:00<00:00, 960MB/s] 


In [3]:
# 2 Вариант - для Kaggle
# IMG_DIR = "/kaggle/input/aaaproject/aaa_extracted/aaa_desc_gen_data/images/"
# DATA_CSV = "/kaggle/input/aaa-data-2/full_dataset.csv"

# 3 Вариант - для локального запуска
# IMG_DIR = '/workspace/AAA_project/data/main_dataset/images/'
# DATA_CSV = '/workspace1/experimental/full_dataset.csv'

In [4]:
dataset = pd.read_csv(DATA_CSV)
dataset

,title,attrs,images,category,multiitem_proba,answers,query
0,Туфли Kapika р.[HIDDEN],"Вид одежды: Для девочек, Предмет одежды: Обувь...",['/workspace/AAA_project/data/main_dataset/ima...,Женская обувь,0.459655,['Новые туфельки для девочки ТМ Kapika Верх ко...,Создай краткое и привлекательное описание для ...
1,Кеды adidas superstar детские,"Предмет одежды: Обувь, Вид одежды: Для мальчик...",['/workspace/AAA_project/data/main_dataset/ima...,Мужская обувь,0.156367,"['Продаю кеды адидас, оригинал, покупались в о...",Создай краткое и привлекательное описание для ...
2,Кросовки [HIDDEN] размер для девочки,"Вид одежды: Для девочек, Предмет одежды: Обувь...",['/workspace/AAA_project/data/main_dataset/ima...,Женская обувь,0.479719,['Кросовки для девочки от Загадка в хорошем со...,Создай краткое и привлекательное описание для ...
3,Crocs сабо детские35-[HIDDEN] / [HIDDEN]-[HIDD...,"Вид одежды: Для девочек, Предмет одежды: Обувь...",['/workspace/AAA_project/data/main_dataset/ima...,Женская обувь,0.178559,"['Состояние отличное, дефект единственный на ф...",Создай краткое и привлекательное описание для ...
4,Ботинки детские Tombi [HIDDEN] размер,"Предмет одежды: Обувь, Вид одежды: Для мальчик...",['/workspace/AAA_project/data/main_dataset/ima...,Мужская обувь,0.326524,"['Ботиночки Tombi,в стиле Timberland,очень удо...",Создай краткое и привлекательное описание для ...
...,...,...,...,...,...,...,...
37436,Кофточки [HIDDEN],"Вид одежды: Для девочек, Предмет одежды: Блузк...",['/workspace/AAA_project/data/main_dataset/ima...,Женские блузки и рубашки,0.076998,['Кофточки для школы. Черная с жемчугом и синя...,Создай краткое и привлекательное описание для ...
37437,Футболка и толстовка для мальчика 152см,"Предмет одежды: Футболки и майки, Вид одежды: ...",['/workspace/AAA_project/data/main_dataset/ima...,Мужские футболки и майки,0.607732,['Вещи пакетом на мальчика. На рост 152см. Вещ...,Создай краткое и привлекательное описание для ...
37438,Хлопковая фланелевая рубашка hm [HIDDEN],"Предмет одежды: Рубашки, Вид одежды: Для мальч...",['/workspace/AAA_project/data/main_dataset/ima...,Мужские рубашки,0.345218,['Новая хлопковая фланелевая рубашка H&M C бир...,Создай краткое и привлекательное описание для ...
37439,Рубашка для мальчика [HIDDEN],"Предмет одежды: Рубашки, Вид одежды: Для мальч...",['/workspace/AAA_project/data/main_dataset/ima...,Мужские рубашки,0.091761,['Рубашка голубого цвета длина рукава [HIDDEN]...,Создай краткое и привлекательное описание для ...


In [5]:
dataset['answers'] = dataset['answers'].apply(lambda x: ast.literal_eval(x)[1]) # Берем ответы от ГПТ4
dataset['images'] = dataset['images'].apply(lambda x: x.replace('/workspace/AAA_project/data/main_dataset/images/', IMG_DIR))

In [6]:
dataset['answers'][1]

'Продаю детские кеды adidas Superstar белого цвета. Размер [HIDDEN] ([HIDDEN] см по стельке). Классический дизайн с тремя черными полосками и фирменным логотипом на язычке и пятке. Кеды в хорошем состоянии, носились несколько раз. Отличный выбор для маленьких модников, которые ценят стиль и комфорт.'

In [7]:
dataset['images'][1]

"['/content/aaa_extracted/aaa_desc_gen_data/images/29690495353.jpg', '/content/aaa_extracted/aaa_desc_gen_data/images/29690495386.jpg', '/content/aaa_extracted/aaa_desc_gen_data/images/29690495401.jpg', '/content/aaa_extracted/aaa_desc_gen_data/images/29690495408.jpg']"

In [8]:
def replace_hidden(x):
    if isinstance(x, str):
        return x.replace('[HIDDEN]', '__')
    elif isinstance(x, list):
        return [replace_hidden(item) for item in x]
    else:
        return x

dataset = dataset.applymap(replace_hidden)

<ipython-input-8-a73ff2e2ca1e>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dataset = dataset.applymap(replace_hidden)


In [9]:
dataset.drop(columns=['title', 'attrs', 'category', 'multiitem_proba'], inplace=True)
dataset.rename(columns={'images': 'images', 'answers': 'text', 'query':'instruction'}, inplace=True)
dataset

,images,text,instruction
0,['/content/aaa_extracted/aaa_desc_gen_data/ima...,"Туфли Kapika для девочек, размер __ см), в нов...",Создай краткое и привлекательное описание для ...
1,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Продаю детские кеды adidas Superstar белого цв...,Создай краткое и привлекательное описание для ...
2,['/content/aaa_extracted/aaa_desc_gen_data/ima...,"Яркие разноцветные кроссовки для девочки, разм...",Создай краткое и привлекательное описание для ...
3,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Продаются оригинальные детские сабо Crocs в яр...,Создай краткое и привлекательное описание для ...
4,['/content/aaa_extracted/aaa_desc_gen_data/ima...,"Продаются детские ботинки Tombi __ размера, ко...",Создай краткое и привлекательное описание для ...
...,...,...,...
37436,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Продаю стильные кофточки для девочек от бренда...,Создай краткое и привлекательное описание для ...
37437,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Продаю комплект одежды для мальчика на рост __...,Создай краткое и привлекательное описание для ...
37438,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Новая хлопковая фланелевая рубашка H&M для мал...,Создай краткое и привлекательное описание для ...
37439,['/content/aaa_extracted/aaa_desc_gen_data/ima...,Продаётся новая голубая рубашка для мальчика. ...,Создай краткое и привлекательное описание для ...


In [10]:
def resize_image(image_path, max_size=512):
    try:
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        width, height = image.size
        if max(width, height) > max_size:
            if width > height:
                new_width = max_size
                new_height = int(height * max_size / width)
            else:
                new_height = max_size
                new_width = int(width * max_size / height)
            image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

        return image
    except Exception as e:
        print(f"Ошибка при обработке изображения {image_path}: {e}")
        return None

In [11]:
def convert_dataset_to_unsloth_format(dataset, num_samples=None, max_images_per_sample=None, image_size=512):
    if num_samples is not None:
        dataset_slice = dataset.head(num_samples)
        print(f"Обрабатываем первые {num_samples} строк из {len(dataset)}")
    else:
        dataset_slice = dataset
        print(f"Обрабатываем весь датасет: {len(dataset)} строк")

    converted_data = []
    for idx, row in dataset_slice.iterrows():
        try:
            if isinstance(row['images'], str):
                image_paths = ast.literal_eval(row['images'])
            else:
                image_paths = row['images']
            if max_images_per_sample is not None and len(image_paths) > max_images_per_sample:
                image_paths = image_paths[:max_images_per_sample]
                print(f"Ограничено количество изображений до {max_images_per_sample} для строки {idx}")

            images = []
            for img_path in image_paths:
                resized_img = resize_image(img_path, max_size=image_size)
                if resized_img is not None:
                    images.append(resized_img)

            if not images:
                print(f"Пропускаем строку {idx}: нет валидных изображений")
                continue

            user_content = [
                {"type": "text", "text": row['instruction']}
            ]

            for img in images:
                user_content.append({"type": "image", "image": img})

            conversation = {
                "messages": [
                    {
                        "role": "user",
                        "content": user_content
                    },
                    {
                        "role": "assistant",
                        "content": [
                            {"type": "text", "text": row['text']}
                        ]
                    }
                ]
            }

            converted_data.append(conversation)
            if idx % 100 == 0:
                print(f"Обработано {idx + 1} строк...")

        except Exception as e:
            print(f"Ошибка при обработке строки {idx}: {e}")
            continue

    print(f"\nУспешно преобразовано {len(converted_data)} примеров")
    return converted_data

In [12]:
def show_dataset_info(converted_data):
    print(f"Количество примеров: {len(converted_data)}")

    first_example = converted_data[0]
    user_msg = first_example['messages'][0]

    text_items = [item for item in user_msg['content'] if item['type'] == 'text']
    image_items = [item for item in user_msg['content'] if item['type'] == 'image']

    print(f"Инструкция: {text_items[0]['text']}")
    print(f"Количество изображений в первом примере: {len(image_items)}")
    if image_items:
        img = image_items[0]['image']
        print(f"Размер первого изображения: {img.size}")

    assistant_msg = first_example['messages'][1]
    print(f"Ответ: {assistant_msg['content'][0]['text']}")

In [13]:
EVAL_NUM = len(dataset) // 10  # 10% от общего числа строк для валидации

eval_dataset = dataset[-EVAL_NUM:]  # Последние строки для валидации
train_dataset = dataset[:-EVAL_NUM]  # Остальные строки для обучения
train_dataset = dataset[0:1001]

In [14]:
eval_data = convert_dataset_to_unsloth_format(
    eval_dataset,
    num_samples=None,
    max_images_per_sample=None, # Без ограничения на количество изображений
    image_size=256           # надо 512
)

Обрабатываем весь датасет: 3744 строк
Обработано 33701 строк...
Обработано 33801 строк...
Обработано 33901 строк...
Обработано 34001 строк...
Обработано 34101 строк...
Обработано 34201 строк...
Обработано 34301 строк...
Обработано 34401 строк...
Обработано 34501 строк...
Обработано 34601 строк...
Обработано 34701 строк...
Обработано 34801 строк...
Обработано 34901 строк...
Обработано 35001 строк...
Обработано 35101 строк...
Обработано 35201 строк...
Обработано 35301 строк...
Обработано 35401 строк...
Обработано 35501 строк...
Обработано 35601 строк...
Обработано 35701 строк...
Обработано 35801 строк...
Обработано 35901 строк...
Обработано 36001 строк...
Обработано 36101 строк...
Обработано 36201 строк...
Обработано 36301 строк...
Обработано 36401 строк...
Обработано 36501 строк...
Обработано 36601 строк...
Обработано 36701 строк...
Обработано 36801 строк...
Обработано 36901 строк...
Обработано 37001 строк...
Обработано 37101 строк...
Обработано 37201 строк...
Обработано 37301 строк...


In [15]:
converted_dataset = convert_dataset_to_unsloth_format(
    train_dataset,
    num_samples=None,
    max_images_per_sample=None, # Без ограничения на количество изображений
    image_size=256           # надо 512
)

show_dataset_info(converted_dataset)

Обрабатываем весь датасет: 1001 строк
Обработано 1 строк...
Обработано 101 строк...
Обработано 201 строк...
Обработано 301 строк...
Обработано 401 строк...
Обработано 501 строк...
Обработано 601 строк...
Обработано 701 строк...
Обработано 801 строк...
Обработано 901 строк...
Обработано 1001 строк...

Успешно преобразовано 1001 примеров
Количество примеров: 1001
Инструкция: Создай краткое и привлекательное описание для объявления о продаже товара или ассортимента товаров. Оно будет размещено на e-commerce платформе. Название товара: Туфли Kapika р.__. Атрибуты товара: Вид одежды: Для девочек, Предмет одежды: Обувь, Состояние: Новый, Вид объявления: Товар приобретён на продажу, Цвет: Синий, Размер: __ см), Бренд одежды: Kapika, Вид обуви: Туфли и босоножки, Тип обуви: Туфли. Категория товара: Женская обувь. Вероятность, что в объявлении много товаров: 0.46. Опиши товар естественно, без списков и структуры, на русском языке. Обязательно упомянуть характеристи товара, которые ты узнал.
Кол

In [16]:
converted_dataset[1]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Создай краткое и привлекательное описание для объявления о продаже товара или ассортимента товаров. Оно будет размещено на e-commerce платформе. Название товара: Кеды adidas superstar детские. Атрибуты товара: Предмет одежды: Обувь, Вид одежды: Для мальчиков, Состояние: Б/у, Вид объявления: Продаю своё, Цвет: Белый, Размер: __ (__ см), Бренд одежды: Adidas, Вид обуви: Кроссовки и кеды, Тип обуви: Кеды. Категория товара: Мужская обувь. Вероятность, что в объявлении много товаров: 0.16. Опиши товар естественно, без списков и структуры, на русском языке. Обязательно упомянуть характеристи товара, которые ты узнал.'},
    {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=144x256>},
    {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=144x256>},
    {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=144x256>},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=

Формат данных удовлетворяет требованиям от Unsloth:

```python
[
{ "role": "user",
  "content": [{"type": "text",  "text": Q}, {"type": "image", "image": image} ]
},
{ "role": "assistant",
  "content": [{"type": "text",  "text": A} ]
},
]
```

## Настройка окружения

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

## Загрузка модели

In [ ]:
from unsloth import FastVisionModel
import torch

fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

Теперь мы добавляем LoRA адаптеры для параметрически эффективного файнтюнинга - это позволяет нам эффективно обучать только 1% от всех параметров.

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False если не файнтюним слои зрения
    finetune_language_layers   = True, # False если не файнтюним языковые слои
    finetune_attention_modules = True, # False если не файнтюним слои внимания
    finetune_mlp_modules       = True, # False если не файнтюним MLP слои

    r = 16,           # Чем больше, тем выше точность, но может переобучиться
    lora_alpha = 16,  # Рекомендуется alpha == r как минимум
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # Возможно использовать стабилизированный по рангу LoRA
    loftq_config = None, # или LoftQ
    # target_modules = "all-linear", # Опционально! Можно указать список если нужно
)

## Модель до LoRA

Давайте сначала посмотрим, что выдает модель на первом примере до файнтюнинга

In [ ]:
FastVisionModel.for_inference(model)

example = converted_dataset[2]  # Берем третий пример
user_message = example['messages'][0]
assistant_message = example['messages'][1]

images = [item['image'] for item in user_message['content'] if item['type'] == 'image']
instruction = [item['text'] for item in user_message['content'] if item['type'] == 'text'][0]
reference_answer = assistant_message['content'][0]['text']

print(f"Инструкция:")
print(f"   {instruction}")
print(f"\nКоличество изображений: {len(images)}")

print(f"Размеры изображений:")
for i, img in enumerate(images):
    print(f"   Изображение {i+1}: {img.size}")

if images:
    num_images = len(images)
    cols = min(3, num_images)
    rows = math.ceil(num_images / cols)

    fig_width = cols * 4
    fig_height = rows * 4
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))

    if num_images == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if isinstance(axes, list) else [axes]
    else:
        axes = axes.flatten()

    for i, img in enumerate(images):
        axes[i].imshow(img)
        axes[i].set_title(f'Изображение {i+1}\nРазмер: {img.size}', fontsize=10)
        axes[i].axis('off')

    for i in range(num_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

print(f"\nЭталонный ответ:")
print(f"   {reference_answer}")

print(f"\nОтвет модели ДО обучения:")

content = []
for img in images:
    content.append({"type": "image"})
content.append({"type": "text", "text": instruction})

messages = [
    {"role": "user", "content": content}
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    images,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("   ", end="")
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

## Обучение

In [ ]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 5, # было 30
        # num_train_epochs = 1, # Используйте это вместо max_steps для полного обучения
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        #max_seq_length = 2048,
        max_seq_length = 4096,  # Увеличено для множественных изображений

    ),
)

In [ ]:
# @title Показать текущие статистики памяти
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Показать финальные статистики памяти и времени
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Инференс"></a>
### Инференс
Давайте запустим модель! Вы можете изменить инструкцию и входные данные - оставьте вывод пустым!

Мы используем `min_p = 0.1` и `temperature = 1.5`. Прочитайте этот [Tweet](https://x.com/menhguin/status/1826132708508213629) для получения дополнительной информации о том, почему.

In [ ]:
FastVisionModel.for_inference(model)

example = converted_dataset[2]
user_message = example['messages'][0]
assistant_message = example['messages'][1]

images = [item['image'] for item in user_message['content'] if item['type'] == 'image']
instruction = [item['text'] for item in user_message['content'] if item['type'] == 'text'][0]
reference_answer = assistant_message['content'][0]['text']

print(f"Инструкция: {instruction}")
print(f"Количество изображений: {len(images)}")
print(f"Эталонный ответ: {reference_answer}")

if images:
    num_images = len(images)
    cols = min(3, num_images)
    rows = math.ceil(num_images / cols)

    fig_width = cols * 4
    fig_height = rows * 4
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))

    if num_images == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if isinstance(axes, list) else [axes]
    else:
        axes = axes.flatten()

    for i, img in enumerate(images):
        axes[i].imshow(img)
        axes[i].set_title(f'Изображение {i+1}\nРазмер: {img.size}', fontsize=10)
        axes[i].axis('off')

    for i in range(num_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

print(f"\nОтвет обученной модели:")

content = []
for img in images:
    content.append({"type": "image", "image": img})
content.append({"type": "text", "text": instruction})

messages = [{"role": "user", "content": content}]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    images,
    input_text,
    add_special_tokens = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(**inputs,
                   streamer = text_streamer,
                   max_new_tokens = 128,
                   min_new_tokens = 32,
                   top_k = 50,
                   use_cache = True,
                   temperature = 0.7,
                   min_p = 0.1,
                   early_stopping = False
                   )

<a name="Сохранение"></a>
## Сохранение и выгрузка из сохранения
Чтобы сохранить финальную модель как LoRA адаптеры, используйте либо `push_to_hub` от Huggingface для онлайн сохранения, либо `save_pretrained` для локального сохранения.

**[ПРИМЕЧАНИЕ]** Это сохраняет ТОЛЬКО LoRA адаптеры, а не полную модель.

In [ ]:
model.save_pretrained("aaa_proj", save_embedding_layers=True)
tokenizer.save_pretrained("aaa_proj",  save_embedding_layers=True)

In [ ]:
model.push_to_hub("moxeeeem/aaa_proj", token=HF_TOKEN,  save_embedding_layers=True)
tokenizer.push_to_hub("moxeeeem/aaa_proj", token=HF_TOKEN,  save_embedding_layers=True)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
#pip install unsloth torch torchvision torchaudio transformers accelerate bitsandbytes peft datasets huggingface_hub

from unsloth import FastVisionModel
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


model, tokenizer = FastVisionModel.from_pretrained(
    model_name = "./aaa_proj",  # локальный вариант
    # model_name = "moxeeeem/aaa_proj",  # или из Hugging Face
    load_in_4bit = True,
)

FastVisionModel.for_inference(model)

In [ ]:
example = converted_dataset[0]
user_message = example['messages'][0]
assistant_message = example['messages'][1]

images = [item['image'] for item in user_message['content'] if item['type'] == 'image']
instruction = [item['text'] for item in user_message['content'] if item['type'] == 'text'][0]
reference_answer = assistant_message['content'][0]['text']

print(f"Инструкция: {instruction}")
print(f"Количество изображений: {len(images)}")
print(f"Эталонный ответ: {reference_answer}")
print(f"\nОтвет загруженной модели:")

if images:
    num_images = len(images)
    cols = min(3, num_images)
    rows = math.ceil(num_images / cols)

    fig_width = cols * 4
    fig_height = rows * 4
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))

    if num_images == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if isinstance(axes, list) else [axes]
    else:
        axes = axes.flatten()

    for i, img in enumerate(images):
        axes[i].imshow(img)
        axes[i].set_title(f'Изображение {i+1}\nРазмер: {img.size}', fontsize=10)
        axes[i].axis('off')

    for i in range(num_images, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

content = []
for img in images:
    content.append({"type": "image", "image": img})
content.append({"type": "text", "text": instruction})

messages = [{"role": "user", "content": content}]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    images,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

# официальный вариант
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

'''
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, min_new_tokens = 32, top_k = 50,
                   use_cache = True, temperature = 0.7, min_p = 0.1,  early_stopping = False)
'''

## Оценка на валидационном датасете

Валидационный датасет уже был подготовлен раннее (eval_data)

In [ ]:
!pip install rouge nltk scikit-learn matplotlib seaborn sentence-transformers transformers

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
import torch
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [ ]:
def preprocess_text(text):
    """Предобработка текста для метрик"""
    text = re.sub(r'\s+', ' ', text.strip())
    text = text.lower()
    return text

def calculate_bleu_score(reference, prediction):
    """Вычисляет BLEU score"""
    try:
        ref_tokens = reference.split()
        pred_tokens = prediction.split()
        smoothing = SmoothingFunction().method1

        bleu_scores = {}
        for n in range(1, 3):  # только BLEU-1 и BLEU-2
            weights = tuple([1/n] * n + [0] * (4-n))
            score = sentence_bleu([ref_tokens], pred_tokens, weights=weights, smoothing_function=smoothing)
            bleu_scores[f'BLEU-{n}'] = score

        return bleu_scores
    except:
        return {'BLEU-1': 0, 'BLEU-2': 0}

def calculate_rouge_scores(reference, prediction):
    """Вычисляет ROUGE scores"""
    try:
        rouge = Rouge()
        scores = rouge.get_scores(prediction, reference)[0]
        return {
            'ROUGE-1-F': scores['rouge-1']['f'],
            'ROUGE-L-F': scores['rouge-l']['f'],
        }
    except:
        return {'ROUGE-1-F': 0, 'ROUGE-L-F': 0}

def calculate_length_metrics(reference, prediction):
    """Вычисляет метрики длины"""
    ref_len = len(reference.split())
    pred_len = len(prediction.split())

    length_ratio = pred_len / ref_len if ref_len > 0 else 0
    length_diff = abs(pred_len - ref_len)

    return {
        'ref_length': ref_len,
        'pred_length': pred_len,
        'length_ratio': length_ratio,
        'length_diff': length_diff
    }

def evaluate_model_on_validation(model, tokenizer, eval_data, max_samples=50):
    """Оценка модели на валидационном датасете"""

    eval_subset = eval_data[:max_samples]
    print(f"Оцениваем модель на {max_samples} примерах из {len(eval_data)}")

    all_metrics = []
    predictions = []
    references = []

    FastVisionModel.for_inference(model)

    for idx, example in enumerate(eval_subset):
        try:
            user_message = example['messages'][0]
            assistant_message = example['messages'][1]

            images = [item['image'] for item in user_message['content'] if item['type'] == 'image']
            instruction = [item['text'] for item in user_message['content'] if item['type'] == 'text'][0]
            reference = assistant_message['content'][0]['text']

            # Генерируем предсказание
            content = []
            for img in images:
                content.append({"type": "image", "image": img})
            content.append({"type": "text", "text": instruction})

            messages = [{"role": "user", "content": content}]

            input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
            inputs = tokenizer(
                images,
                input_text,
                add_special_tokens=False,
                return_tensors="pt",
            ).to("cuda")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=128,
                    min_new_tokens=10,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )

            input_length = inputs['input_ids'].shape[1]
            generated_tokens = outputs[0][input_length:]
            prediction = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

            # Предобработка текстов
            reference_clean = preprocess_text(reference)
            prediction_clean = preprocess_text(prediction)

            # Вычисляем метрики
            example_metrics = {}

            bleu_scores = calculate_bleu_score(reference_clean, prediction_clean)
            example_metrics.update(bleu_scores)

            rouge_scores = calculate_rouge_scores(reference_clean, prediction_clean)
            example_metrics.update(rouge_scores)

            length_metrics = calculate_length_metrics(reference_clean, prediction_clean)
            example_metrics.update(length_metrics)

            example_metrics['example_id'] = idx
            example_metrics['num_images'] = len(images)

            all_metrics.append(example_metrics)
            predictions.append(prediction_clean)
            references.append(reference_clean)

            if (idx + 1) % 10 == 0:
                print(f"Обработано {idx + 1}/{len(eval_subset)} примеров...")

        except Exception as e:
            print(f"Ошибка при обработке примера {idx}: {e}")
            continue

    return all_metrics, predictions, references

In [ ]:
metrics_data, all_predictions, all_references = evaluate_model_on_validation(
    model, tokenizer, eval_data, max_samples=len(eval_data)
)

In [ ]:
def analyze_metrics(metrics_data):
    """Анализ метрик"""
    df = pd.DataFrame(metrics_data)

    print("Статистика метрик:\n")

    main_metrics = ['BLEU-1', 'BLEU-2', 'ROUGE-1-F', 'ROUGE-L-F']

    print("Средние значения:")
    for metric in main_metrics:
        if metric in df.columns:
            mean_val = df[metric].mean()
            std_val = df[metric].std()
            print(f"{metric:15}: {mean_val:.4f} ± {std_val:.4f}")

    print("\nАнализ длины:")
    print(f"Средняя длина референсов:     {df['ref_length'].mean():.2f} слов")
    print(f"Средняя длина предсказаний:   {df['pred_length'].mean():.2f} слов")
    print(f"Соотношение длин:             {df['length_ratio'].mean():.4f}")
    print(f"Разность длин:                {df['length_diff'].mean():.2f} слов")

    return df

In [ ]:
def create_simple_visualizations(df):
    """Простые визуализации"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # BLEU scores
    bleu_cols = ['BLEU-1', 'BLEU-2']
    axes[0, 0].boxplot([df[col].values for col in bleu_cols], labels=bleu_cols)
    axes[0, 0].set_title('BLEU Scores')
    axes[0, 0].grid(True, alpha=0.3)

    # ROUGE scores
    rouge_cols = ['ROUGE-1-F', 'ROUGE-L-F']
    axes[0, 1].boxplot([df[col].values for col in rouge_cols], labels=['ROUGE-1', 'ROUGE-L'])
    axes[0, 1].set_title('ROUGE Scores')
    axes[0, 1].grid(True, alpha=0.3)

    # Соотношение длин
    axes[1, 0].scatter(df['ref_length'], df['pred_length'], alpha=0.6)
    axes[1, 0].plot([0, df['ref_length'].max()], [0, df['ref_length'].max()], 'r--', alpha=0.8)
    axes[1, 0].set_xlabel('Длина референса')
    axes[1, 0].set_ylabel('Длина предсказания')
    axes[1, 0].set_title('Сравнение длин')
    axes[1, 0].grid(True, alpha=0.3)

    # Распределение соотношения длин
    axes[1, 1].hist(df['length_ratio'], bins=20, alpha=0.7, color='orange')
    axes[1, 1].set_xlabel('Соотношение длин')
    axes[1, 1].set_ylabel('Частота')
    axes[1, 1].set_title('Распределение соотношения длин')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return fig

In [ ]:
def show_examples(metrics_data, all_predictions, all_references, eval_data, n_examples=3):
    """Показать примеры"""
    df = pd.DataFrame(metrics_data)

    print("Лучшие примеры по BLEU-1:")
    print("-" * 60)

    best_idx = df.nlargest(n_examples, 'BLEU-1').index
    for i, idx in enumerate(best_idx):
        example = eval_data[idx]
        user_msg = example['messages'][0]
        instruction = [item['text'] for item in user_msg['content'] if item['type'] == 'text'][0]

        print(f"\nПример {i+1}:")
        print(f"BLEU-1: {df.loc[idx, 'BLEU-1']:.4f}")
        print(f"ROUGE-1: {df.loc[idx, 'ROUGE-1-F']:.4f}")
        print(f"Инструкция: {instruction[:80]}...")
        print(f"Референс: {all_references[idx][:80]}...")
        print(f"Предсказание: {all_predictions[idx][:80]}...")

In [ ]:
df_metrics = analyze_metrics(metrics_data)

In [ ]:
fig = create_simple_visualizations(df_metrics)

In [ ]:
show_examples(metrics_data, all_predictions, all_references, eval_data)

In [ ]:
df_final = pd.DataFrame(metrics_data)
print(f"Обработано примеров: {len(df_final)}")
print(f"Средний BLEU-1: {df_final['BLEU-1'].mean():.4f}")
print(f"Средний ROUGE-1-F: {df_final['ROUGE-1-F'].mean():.4f}")
print(f"Средняя точность длины: {1 - abs(df_final['length_ratio'].mean() - 1):.4f}")